## Project Title

### Project Description

Our team will focus on studying the relationship (if any) of the average land temperature (i.e., #2 proposed dataset) and the evolution of population in such land, along with its GDP. Datasets will be pulled from the World Bank and are available in `.csv`, `.xml` and `.xlsx` formats in the following links, respectively:

[Population](https://data.worldbank.org/indicator/SP.POP.TOTL)

[GDP (2015 USD)](https://data.worldbank.org/indicator/NY.GDP.MKTP.KD)

Behavior will be analysed mainly by visual means (such as scatterplots, pyramid populations, etc.) and reported in Markdown in the corresponding .ipynb file. 

> Imported libraries

In [1]:
import pandas as pd
import geopandas
import folium 



## Programming Component



### Reading base dataset



### Combining and cleaning data



### Calculations